In [1]:
import git
from datetime import date
from datetime import timedelta

import os
from bbid import bbid

from pathlib import Path

import string
import random

import tensorflow as tf
import numpy as np
import json
import zipfile
import shutil

import kfp

import urllib

from kale.common import rokutils
from cat_cnn_component.cat_preprocessing import set_label

import requests
import pickle
import filecmp
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.cd import KSDrift, FETDrift
from PIL import Image
from skimage import io
import gc

from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer, Reshape
from alibi_detect.cd.tensorflow import preprocess_drift
from functools import partial
import matplotlib.pyplot as plt
import time

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [3]:
repo = git.Repo('.')
o = repo.remotes.origin
o.pull()
print("Pulled Code from main branch.")

last_week = date.today() - timedelta(7)

In [4]:
commits = list(repo.iter_commits(since=last_week))

In [5]:
code_updated = False

for commit in commits:
    print("Committed by %s on %s with sha %s" % (commit.committer.name, time.strftime("%a, %d %b %Y %H:%M", time.localtime(commit.committed_date)), commit.hexsha))
    print("Commit message: " + str(commit.message))
    print("-----------------------------------------")
    if ' --no-trigger' not in commit.message:
        code_updated = True
        print('Code update detected!')
        break

In [6]:
print("is_code_updated: " + str(code_updated))

is_code_updated: False


In [2]:
print("Downloading new dataset for comparison...")

In [10]:
url = "http://10.0.2.2:8000/prediction/pipeline_dataset"
try:
    os.remove('/home/jovyan/cat-breed-vol-1/new_dataset.zip')
except:
    pass
with open('/home/jovyan/cat-breed-vol-1/new_dataset.zip', 'wb') as out_file:
    content = requests.get(url, stream=True).content
    out_file.write(content)

In [2]:
dataset_updated = not filecmp.cmp('/home/jovyan/cat-breed-vol-1/new_dataset.zip', '/home/jovyan/cat-breed-vol-1/dataset.zip')
print("is_dataset_updated: " + str(dataset_updated))
if dataset_updated is True:
    print('New dataset will overtake as training dataset.')
    os.remove('/home/jovyan/cat-breed-vol-1/dataset.zip')
    shutil.rmtree('/home/jovyan/cat-breed-vol-1/dataset/')
    with zipfile.ZipFile('/home/jovyan/cat-breed-vol-1/new_dataset.zip', 'r') as zip_ref:
        zip_ref.extractall('/home/jovyan/cat-breed-vol-1/dataset/')
    os.rename('/home/jovyan/cat-breed-vol-1/new_dataset.zip', '/home/jovyan/cat-breed-vol-1/dataset.zip')
    new_label = set_label()
    print('New dataset labels: ' + str(new_label))

In [3]:
Path.isdir = Path.is_dir

In [4]:
if os.path.exists('/home/jovyan/cat-breed-vol-1/drift/'):
    shutil.rmtree('/home/jovyan/cat-breed-vol-1/drift/')

In [5]:
labels = set_label()

labels: ['Bengal', 'Egyptian Mau', 'Persian', 'Siamese', 'Tabby']


In [6]:
if dataset_updated is False:
    NUMBER_OF_IMAGES_PER_CLASS = 12
    print('Downloading '+str(NUMBER_OF_IMAGES_PER_CLASS)+' images for each label from Bing.')
    count = 0
    for label in labels:
        query_string = label + ' cat breed'
        output_dir = os.path.join("/home/jovyan/cat-breed-vol-1/drift")
        bbid(query_string, os.path.join(output_dir, label), NUMBER_OF_IMAGES_PER_CLASS)
        count+=1        
    
    dataset = []
    y_dataset = []
    label_names = labels
    
    print('Testing for data drift.')
    
    for label in label_names:
        path = "/home/jovyan/cat-breed-vol-1/drift/" + label + "/"
        for item in os.listdir(path):
            try:
                if item.split('.')[1] == 'jpg':
                    image = io.imread(path+item)
                    image = Image.fromarray(image, 'RGB')
                    image = image.resize((200, 200))
                    dataset.append(np.array(image))
                    y_dataset.append(label_names.index(label))
            except Exception as e:
                os.remove(path+item)
                pass

    dataset = np.array(dataset)
    y_dataset = np.array(y_dataset)
    
    print("Show 9 sample images from drift detection dataset.")
    f,a = plt.subplots(nrows=4, ncols=3,figsize=(13, 7), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(a.flat):
        ax.imshow(dataset[i])
        ax.set_title(label_names[y_dataset[i]])
    
    plt.tight_layout()
    plt.show()
    
    cd = load_detector('/home/jovyan/cat-breed-vol-1/drift_detection_model/KSDrift/')
    preds = cd.predict(dataset)
    data_drifted = bool(preds['data']['is_drift'])
    
    print("is_data_drifted: " + str(data_drifted))
else:
    print("Dataset updated, skipped data drift detection.")
    data_drifted = False

 OK : 306059656_c4fafc4a87_o.jpg
 OK : bengalcat-56a2bcd95f9b58b7d0cdf798.jpg
 OK : image
 OK : lying_bengal_cat_widescreen_wallpape.jpg
 OK : 25%2BGorgeous%2BBengal%2BCat%2BBreed.jpg
 OK : 435a50f155c74f04644c7d48174b415e.jpg
 OK : 25%2BGorgeous%2BBengal%2BCat%2BBreed-1.jpg
 OK : a453d75a3f2e8b000fb6b7188d342238.jpg
 OK : 2308642374_57489bb2ce_o-57b742c15f9b.jpg
 OK : cc87feaa37aff4af7bee0eececc94ba5.jpg
 OK : image-1
 OK : 3931eb090308eeaa58719fe66b9926c3.jpg
1
 OK : 7705350_f496.jpg
 OK : bengal-cat-not-a-pet-for-everyone.jpg
 OK : bengal1.jpg
 OK : DSC01651.jpg
 OK : bengal-cat-03.jpg
 OK : 8bd4e0252c09ed243bd329bde4f6d6f0.jpg
 OK : 407a0798b608c6fad50ad67dc56966c8.jpg
 OK : 59c8741a1dace0395133e9f11b26d7a6.jpg
 OK : 623b21e78b120d015c733b4a48d90365.jpg
 OK : 4734659828_39868fe4cd_z.jpg
 OK : cd575f0d7afa1027ab86b193ba3749db.jpg
 OK : Egyptian-Mau-Cat-Picture.jpg
FAIL: egyptian mau 2.jpg
SKIP: Invalid image, not saving Egyptian-mau-close.jpg
SKIP: Invalid image, not saving the-egyp

In [8]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

if dataset_updated is False:
    print('Testing for concept drift.')
    dataset = []
    y_dataset = []
    label_names = set_label()
    for label in label_names:
        count = 0
        path = "/home/jovyan/cat-breed-vol-1/dataset/dataset/images/" + label + "/"
        items = os.listdir(path)
        random.Random(123).shuffle(items)
        for item in items[0:500]:
            try:
                if item.split('.')[1] == 'jpg':
                    image = io.imread(path+item)
                    image = Image.fromarray(image, 'RGB')
                    image = image.resize((200, 200))
                    dataset.append(np.array(image))
                    y_dataset.append(label_names.index(label))
            except:
                pass
                
    dataset = np.array(dataset)
    y_dataset = np.array(y_dataset)
    
    dataset, y_dataset = unison_shuffled_copies(dataset, y_dataset)
    model = tf.keras.models.load_model('/home/jovyan/saved_model_tmp/')
    
    preds = model.predict(dataset[0:500])
    res = preds.argmax(axis=1)
    res = (res==y_dataset[0:500]).astype(int)
    cd_fet = FETDrift(res, p_val=0.05, alternative='less')
    
    print('Using dataset downloaded from is_data_drifted.')
    dataset = []
    y_dataset = []
    label_names = set_label()
    for label in label_names:
        path = "/home/jovyan/cat-breed-vol-1/drift/" + label + "/"
        for item in os.listdir(path):
            try:
                if item.split('.')[1] == 'jpg':
                    image = io.imread(path+item)
                    image = Image.fromarray(image, 'RGB')
                    image = image.resize((200, 200))
                    dataset.append(np.array(image))
                    y_dataset.append(label_names.index(label))
            except:
                os.remove(path+item)
                pass
    dataset = np.array(dataset)
    y_dataset = np.array(y_dataset)
    preds = model.predict(dataset)
    res = preds.argmax(axis=1)
    res = (res==y_dataset).astype(int)
    
    preds = cd_fet.predict(res)
    concept_drifted = bool(preds['data']['is_drift'])
    print("is_concept_drifted: " + str(concept_drifted))
else:
    print("Dataset updated, skipped concept drift detection.")
    concept_drifted = False

labels: ['Bengal', 'Egyptian Mau', 'Persian', 'Siamese', 'Tabby']
Error: ['Egyptian_Mau_129', 'jpg']
labels: ['Bengal', 'Egyptian Mau', 'Persian', 'Siamese', 'Tabby']
Error: ['image']
Error: ['The-Siamese-Cat-5', 'jpg']
is_concept_drifted: False


In [9]:
try:
    print('Getting user reports to see if retraining needed.')
    response = requests.get(
             'http://10.0.2.2:8000/prediction/is_retraining')
    user_training_needed = json.loads(response.text)['training_needed']
except:
    print("Error connecting to frontend server, default to False.")
    user_training_needed = False
print("is_user_report_retraining: " + str(user_training_needed))

is_user_report_retraining: False


In [10]:
dataset_mutated = False
if data_drifted or concept_drifted:
    print('Model drifted, adding drift data to training dataset.')
    labels = set_label()
    for label in labels:
        for file in os.listdir('/home/jovyan/cat-breed-vol-1/drift/'+ label):
            try:
                os.rename(os.path.join('/home/jovyan/cat-breed-vol-1/drift',label,file), os.path.join('/home/jovyan/cat-breed-vol-1/dataset/dataset/images', label, file))
            except:
                pass
    dataset_mutated = True

In [11]:
if user_training_needed:
    print('User data retraining needed, download and add user data to training dataset.')
    url = 'http://10.0.2.2:8000/prediction/new_dataset'
    location = ''
    try:
        os.remove("/home/jovyan/cat-breed-vol-1/dataset_user.zip")
        shutil.rmtree('/home/jovyan/cat-breed-vol-1/dataset_user/')
    except:
        pass
    urllib.request.urlretrieve(url, '/home/jovyan/cat-breed-vol-1/dataset_user.zip')
    with zipfile.ZipFile('/home/jovyan/cat-breed-vol-1/dataset_user.zip', 'r') as zip_ref:
        zip_ref.extractall('/home/jovyan/cat-breed-vol-1/dataset_user/')
    
    for label in labels:
        for file in os.listdir(os.path.join('/home/jovyan/cat-breed-vol-1/dataset_user/dataset/images', label)):
            os.rename(os.path.join('/home/jovyan/cat-breed-vol-1/dataset_user/dataset/images', label, file), os.path.join('/home/jovyan/cat-breed-vol-1/dataset/dataset/images', label, file))
    dataset_mutated = True

In [12]:
if dataset_mutated is True or dataset_updated is True:
    print("Snapshoting pvc containing dataset.")
    BUCKET_NAME='cat-dataset-bucket'
    volume_name='cat-breed-vol-1-iwqncg7e2'

    task_info = rokutils.snapshot_pvc(volume_name,
                                      bucket=BUCKET_NAME,
                                      wait=True)
    task = rokutils.get_task(task_info["task"]["id"],
                             bucket=BUCKET_NAME)
    print("Snapshot completed. task_id = " + str(task_info["task"]["id"]))

In [13]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
def image_example(image_string, label):
    image_shape = tf.io.decode_jpeg(image_string).shape

    feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [15]:
labels = set_label()

labels: ['Bengal', 'Egyptian Mau', 'Persian', 'Siamese', 'Tabby']


In [16]:
if dataset_updated is True or dataset_mutated is True:
    print("Generating tfrecord.")
    try:
        os.remove('/home/jovyan/cat-breed-vol-1/images-cat.tfrecords')
    except:
        pass

    record_file = '/home/jovyan/cat-breed-vol-1/images-cat.tfrecords'
    with tf.io.TFRecordWriter(record_file) as writer:
        for label in labels:
            for filename in os.listdir('/home/jovyan/cat-breed-vol-1/dataset/dataset/images/'+ label):
                b_dir = '/home/jovyan/cat-breed-vol-1/dataset/dataset/images/'+ label +'/'+filename
                image_string = open(b_dir, 'rb').read()
                try:
                    tf_example = image_example(image_string, labels.index(label))
                    writer.write(tf_example.SerializeToString())
                except:
                    pass
    print("TFRecord generated.")

In [17]:
print(data_drifted or code_updated or concept_drifted or dataset_mutated)

False


In [18]:
def random_string(size=10, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def listToString(s):
    str1 = " " 
    return (str1.join(s))

if data_drifted or code_updated or concept_drifted or dataset_mutated or dataset_updated:
    client = kfp.Client()
    experiment = client.get_experiment(experiment_name='cat-breed-pipeline-v1')
    pipeline = client.get_pipeline(client.get_pipeline_id('cat-breed-pipeline'))
    job_name = random_string()
    obj = client.run_pipeline(experiment_id=experiment.id, pipeline_id=pipeline.id, job_name=job_name)
    print("Model retraining pipeline triggered, run_id = " + str(obj.id))
    print("Waiting for model retraining pipeline to finish...")
    client.wait_for_run_completion(obj.id, 1800)
    print("Model retraining pipeline run finished.")
    print(str(obj.id) + "run completed")
    try:
        print('Uploading new dataset to frontend.')
        if os.path.exists("/home/jovyan/cat-breed-vol-1/labels_pickled"):
            os.remove("/home/jovyan/cat-breed-vol-1/labels_pickled")
        data = {
        'labels': set_label(),
        }
        with open('/home/jovyan/cat-breed-vol-1/labels_pickled', 'wb') as f:
            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        value = {"name": str(job_name)}
        files = {'tfrecord': open('/home/jovyan/cat-breed-vol-1/images-cat.tfrecords','rb'), 'labels_pickled':open('/home/jovyan/cat-breed-vol-1/labels_pickled', 'rb')}
        r = requests.post('http://10.0.2.2:8000/prediction/download_tfrecord', files=files, data=value)
        print("TFRecord and new labels uploaded to frontend")
    except:
        print('Unable connect to server, No file uploaded.')
#         pass

In [19]:
if dataset_updated or dataset_mutated:
    print('Retraining model for drift detection.')
    dataset = []
    label_names = set_label()
    for label in label_names:
        count = 0
        path = "/home/jovyan/cat-breed-vol-1/dataset/dataset/images/" + label + "/"
        items = os.listdir(path)
        random.Random(123).shuffle(items)
        for item in items[0:500]:
            try:
                if item.split('.')[1] == 'jpg':
                    image = io.imread(path+item)
                    image = Image.fromarray(image, 'RGB')
                    image = image.resize((200, 200))
                    dataset.append(np.array(image))
            except:
                os.remove(path+item)
                pass

    dataset = np.array(dataset)
    
    encoding_dim = 32
    encoder_net = tf.keras.Sequential(
      [
          InputLayer(input_shape=(200, 200, 3)),
          Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
          Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
          Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
          Flatten(),
          Dense(encoding_dim,)
      ]
    )

    preprocess_fn = partial(preprocess_drift, model=encoder_net, batch_size=64)

    cd = KSDrift(dataset, p_val=.05, preprocess_fn=preprocess_fn)
    path = '/home/jovyan/cat-breed-vol-1/drift_detection_model/KSDrift/'
    save_detector(cd, path)
    print('Retraining complete, saved drift detection model at '+ path)